In [ ]:
import datetime
import json
import itertools
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
import glob
import os
from matplotlib import pyplot as plt
from pathlib import Path
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from algo.universe.assets import get_asset_data
from algo.blockchain.utils import timestamp_to_5min
from functools import lru_cache
from definitions import ROOT_DIR

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
@lru_cache()
def get_asset_name(asset_id):
    return get_asset_data(asset_id)['params']['name']

@lru_cache()
def get_decimals(asset_id):
    return get_asset_data(asset_id)['params']['decimals']


def load_from_cache(pattern, filter_pair):

    def gen_data():
        for base_dir in glob.glob(pattern):
            a0,a1 = tuple(int(x) for x in Path(base_dir).name.split('_'))
            
            if filter_pair(a0,a1):
                subdf = pd.concat(pd.read_parquet(fname) for fname in glob.glob(f'{base_dir}/*.parquet')).sort_values(by='time')
                subdf['asset1'] = a0
                subdf['asset2'] = a1
                yield subdf

    df = pd.concat(gen_data())
    df['time_5min'] = timestamp_to_5min(df['time'])
    return df

In [ ]:
cache_name = '20220206'
# Select only pairs with Algo
pattern = f'{ROOT_DIR}/caches/prices/{cache_name}/0_*'

def filter_pair(a1, a2):
    exclude = [31566704, # USDC
               312769, # USDt
                567485181, # LoudDefi 
                  ]
    if a1 in exclude or a2 in exclude:
        return False
    return True


dfp_raw = load_from_cache(pattern, filter_pair)
dfp = dfp_raw.groupby(['time_5min','asset1','asset2']).agg('mean').reset_index()
dfp.head()

In [ ]:
cache_name = '20220206'
# Select only pairs with Algo
pattern = f'{ROOT_DIR}/caches/volumes/{cache_name}/0_*'

def filter_pair(a1, a2):
    stablecoins = [31566704, # USDC
                   312769 # USDt
                  ]
    if a1 in stablecoins or a2 in stablecoins:
        return False
    return True


dfv_raw = load_from_cache(pattern, filter_pair)
dfv_raw['algo_volume'] = abs(dfv_raw['asset1_amount'])
dfv = dfv_raw.groupby(['time_5min','asset1','asset2']).agg('sum').reset_index()
dfv.head()

In [ ]:
def price_norm(subdf, asset_id):
    decimals = get_decimals(asset_id)
    subdf['price_norm'] = subdf['price'] * 10**(decimals-6)
    return subdf

def cumulative_volume(subdf):
    subdf['algo_volume_cumul'] = subdf['algo_volume'].cumsum() / (10**6)
    return subdf

def cumulative_net_volume(subdf):
    subdf['net_algo_volume_cumul'] = subdf['asset1_amount'].cumsum() / (10**6)
    return subdf

keys = ['time_5min','asset1','asset2']
df = dfp.merge(dfv, how='left', on=keys)

df['price'] = df['asset2_reserves']/df['asset1_reserves']
df = df.groupby('asset2').apply(lambda x: price_norm(x,x.name))
df = df.groupby('asset2').apply(cumulative_volume)
df = df.groupby('asset2').apply(cumulative_net_volume)
df['reserves_algo'] = df['asset2_reserves'] / (10**6)
df['net_algo_volume'] = df['asset1_amount'] / (10**6)

df.head()

In [ ]:
keyname = 'asset2'

keys = list(df[keyname].unique())

cols = ('price_norm', 'reserves_algo', 'algo_volume_cumul')#, 'net_algo_volume_cum')

nrows = len(cols)
ncols = len(keys)
f,axss = plt.subplots(nrows, ncols, figsize=(8*ncols,4*nrows), sharex='col')

for j,(asset_id,group) in enumerate(df.groupby([keyname])):
    asset_name = get_asset_name(asset_id)


    for i,col in enumerate(cols):
        ax = axss[i][j]
        ax.plot(group['time_5min'], group[col], label=col);
        ax.set_title(f'{asset_name}, {asset_id}');
        ax.legend();
        ax.grid();

plt.xticks(rotation=35);
plt.savefig('test.pdf')